In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from datetime import datetime
from dateutil import parser
import matplotlib.pyplot as plt
import calendar
from datetime import timedelta

In [2]:
data_df = pd.read_csv('D:/BlueDart/POC/data.csv')
data_df.head()

,Run Type,NCARNETNO,DFLIGHTDT,CFLIGHTNO,Route,CORG,O_City_type,Org-Reg,CDST,D_City_type,...,Sch. Arrival,Actual Arrival,Expected Arrival,O_D,Departure delay,Arrival Delay,day,month,year,date
0,SFC,494301,01-Jan-2018,PJD12A,PJD,ATR,Non-Metro,NORTH,MHP,Metro,...,1/2/2018 9:40,1/2/2018 9:40,1/2/2018 9:40,NORTH-NORTH,0.00,0.00,1,1,2018,1/1/2018
1,SFC,900000004829856,01-Jan-2018,PVH11,PVH,PNW,Non-Metro,WEST1,HBW,Non-Metro,...,1/1/2018 14:25,1/1/2018 17:35,1/1/2018 14:25,WEST1-SOUTH2,0.00,3.17,1,1,2018,1/1/2018
2,SFC,900000004829641,01-Jan-2018,AMD11,AMD,AMH,Non-Metro,NORTH,DIA,Non-Metro,...,1/1/2018 12:43,1/1/2018 13:04,1/1/2018 12:43,NORTH-NORTH,0.00,0.35,1,1,2018,1/1/2018
3,SFC,494218,01-Jan-2018,DRD12,DRD,DRN,Non-Metro,NORTH,MHP,Metro,...,1/2/2018 3:30,1/2/2018 3:30,1/2/2018 3:30,NORTH-NORTH,0.00,0.00,1,1,2018,1/1/2018
4,SFC,900000004830087,01-Jan-2018,JNK11,JNK,JNS,Non-Metro,NORTH,KWH,Non-Metro,...,1/1/2018 23:00,1/1/2018 22:35,1/1/2018 22:19,NORTH-NORTH,-0.68,-0.42,1,1,2018,1/1/2018


In [3]:
data_df.columns

Index(['Run Type', 'NCARNETNO', 'DFLIGHTDT', 'CFLIGHTNO', 'Route', 'CORG',
       'O_City_type', 'Org-Reg', 'CDST', 'D_City_type', 'Dst_Reg', 'Distance',
       'CVEHTYPE', 'TYPE', 'Vendor', 'Capacity', 'Consdwt', 'Pcs',
       'Util_conwt', 'Crit', 'NHIRE_AMT', 'FS', 'Incentive/Penalty',
       'NOTHERSAMT', 'NDISCOUNT', 'NBDEDEDUCT', 'Intra/Inter', 'CCONTTYPE',
       'Sch/NSch/Spl', 'Sch. Departure', 'Act. Departure', 'Sch. Arrival',
       'Actual Arrival', 'Expected Arrival', 'O_D', 'Departure delay',
       'Arrival Delay', 'day', 'month', 'year', 'date'],
      dtype='object')

In [4]:
# Converting to datetime
col_names = ['DFLIGHTDT','Sch. Departure', 'Act. Departure','Sch. Arrival', 'Expected Arrival','Actual Arrival','date']
data_df[col_names] = data_df[col_names].applymap(lambda x: parser.parse(x) if pd.notnull(x) else x)

In [5]:
data = data_df.copy()

In [6]:
print(data.shape)

# Departures and Arrivals
print("Nulls in Actual Arrivals: ", pd.isnull(data['Actual Arrival']).sum())

# # Imputing nulls in Actual Arrivals by Expected Arrivals
# data['Actual Arrival'] = np.where(pd.isnull(data['Actual Arrival']), data['Expected Arrival'], data['Actual Arrival'])

# Removing data with nulls in Actual Arrival
data = data.loc[pd.notnull(data['Actual Arrival'])]
print(data.shape)

# Removing nulls and 0's from Distance
data = data.loc[pd.notnull(data['Distance'])]
data = data.loc[(data['Distance'] != 0)]

# Removing nulls from CVEHICLE TYPE
data = data.loc[pd.notnull(data['CVEHTYPE'])]
print(data.shape)

# Removing P2P from Run Type
data = data.loc[data['Run Type'] != 'P2P']
print(data.shape)

print(((data['Consdwt'] == 0)&(data['Pcs'] == 0)).sum())
# Removing 0 Weight Shipments
data = data.loc[data['Consdwt'] != 0]
print(data.shape)

# Removing 0 from Number of Shipments
data = data.loc[data['Pcs'] != 0]
print(data.shape)

cost_cols = ['NHIRE_AMT', 'FS', 'Incentive/Penalty','NOTHERSAMT', 'NDISCOUNT', 'NBDEDEDUCT']
sum(data[[i for i in data.columns if i not in cost_cols]].apply(lambda x: pd.isnull(x).sum()))

(94675, 41)
Nulls in Actual Arrivals:  1801
(92874, 41)
(90659, 41)
(90498, 41)
381
(90083, 41)
(90082, 41)


0

In [7]:
col_names = ['DFLIGHTDT','Sch. Departure', 'Act. Departure','Sch. Arrival', 'Expected Arrival','Actual Arrival']
for col in col_names:
    print(col, set((map(lambda x: x.year, data[col]))))
    
data['Departure Year'] = list(map(lambda x: x.year, data['Act. Departure']))
data['Actual Arrival Year'] = list(map(lambda x: x.year, data['Actual Arrival']))
data.loc[(data['Departure Year'] == 2017)|(data['Actual Arrival Year'] !=2018), col_names]

DFLIGHTDT {2018}
Sch. Departure {2017, 2018}
Act. Departure {2017, 2018}
Sch. Arrival {2018}
Expected Arrival {2018}
Actual Arrival {2017, 2018, 2019}


,DFLIGHTDT,Sch. Departure,Act. Departure,Sch. Arrival,Expected Arrival,Actual Arrival
14,2018-01-01,2018-01-01 22:00:00,2018-01-01 21:30:00,2018-01-02 05:15:00,2018-01-02 04:45:00,2017-12-27 05:45:00
114,2018-01-01,2017-12-31 22:00:00,2017-12-31 22:00:00,2018-01-01 02:00:00,2018-01-01 02:00:00,2018-01-01 02:00:00
191,2018-01-01,2017-12-31 19:30:00,2017-12-31 19:30:00,2018-01-02 05:30:00,2018-01-02 05:30:00,2018-01-02 20:35:00
195,2018-01-01,2018-01-01 00:05:00,2017-12-31 21:30:00,2018-01-01 12:30:00,2018-01-01 09:55:00,2018-01-01 09:25:00
225,2018-01-01,2017-12-31 21:30:00,2017-12-31 21:30:00,2018-01-02 09:30:00,2018-01-02 09:30:00,2018-01-02 18:35:00
248,2018-01-01,2018-01-01 00:30:00,2017-12-31 14:30:00,2018-01-02 12:30:00,2018-01-02 02:30:00,2018-01-02 10:00:00
249,2018-01-01,2017-12-31 23:15:00,2017-12-31 23:15:00,2018-01-02 11:15:00,2018-01-02 11:15:00,2018-01-02 14:05:00
252,2018-01-01,2017-12-31 15:00:00,2017-12-31 15:00:00,2018-01-02 03:00:00,2018-01-02 03:00:00,2018-01-02 15:35:00
8001,2018-01-22,2018-01-22 06:00:00,2018-01-22 06:00:00,2018-01-22 10:17:00,2018-01-22 10:17:00,2019-01-22 11:00:00


In [8]:
# Drop rows with more than 10 days of delay
# These are deemed anomalous
data['Delay'] = data['Actual Arrival'] - data['Sch. Arrival']
data['Delay'] = data['Delay'].apply(lambda x: np.round(x.total_seconds()/3600))
anomalous_delay = data.loc[(data['Delay'] > 10*24)|(data['Delay'] < -10*24), col_names + ['Delay']].index
print(len(anomalous_delay))
data = data.drop(anomalous_delay)

print(data.shape)

6
(90076, 44)


In [9]:
data['Delay_Flag'] = [1 if x > 0 else 0 for x in data['Delay']]
(data['Delay'] > 0).sum(), (data['Delay'] < 0).sum(), (data['Delay'] == 0).sum()

(38602, 35194, 16280)

In [10]:
# data.to_excel('D:/BlueDart/POC/Data_v2.xlsx', index = False)

In [11]:
# data.to_csv('D:/BlueDart/POC/Data_v2.csv', index = False)

In [12]:
# Run Type
data['Run Type'] = list(map(lambda x: x.upper(), data['Run Type']))
print(Counter(data['Run Type']))

Counter({'SFC': 80857, 'SPEED': 6938, 'AIRNET': 2281})


In [13]:
# Month
data['Month'] = data['DFLIGHTDT'].apply(lambda x: x.month)
# Weekday
data['Weekday'] = data['DFLIGHTDT'].apply(lambda x: calendar.day_name[x.weekday()])
# Time of Departure
intervals = pd.IntervalIndex.from_tuples([(0,4),(4,8),(8,12),(12,16),(16,20),(20,24)],closed='left')
data['Time_Interval'] = pd.cut(data['Sch. Departure'].dt.hour, intervals)

In [14]:
# City Type
data['O_D_city_type'] = data['O_City_type'] + '/' + data['D_City_type']
pd.crosstab(index=data['Delay_Flag'],columns=data['O_D_city_type']).apply(lambda x: x/x.sum(), axis=0)

O_D_city_type,Metro/Metro,Metro/Non-Metro,Non-Metro/Metro,Non-Metro/Non-Metro
Delay_Flag,,,,
0,0.477477,0.545029,0.600722,0.65356
1,0.522523,0.454971,0.399278,0.34644


In [15]:
len(set(data['Route'])), len(set(data['Vendor']))

(438, 369)

In [16]:
# Route
route_df = data.groupby(['O_D_city_type','Route']).agg({'Route':'count','Delay_Flag':'sum'})
route_df['1'] = route_df['Delay_Flag']/route_df['Route']
route_df['0'] = 1 - route_df['1']
r_df = route_df.loc[(route_df['Route'] > 350) & (np.abs(route_df['1'] - route_df['0']) > 0.4)]
print(r_df.shape)
r_df

(20, 4)


Route  Delay_Flag         1         0
O_D_city_type       Route                                         
Metro/Metro         BBL        807         716  0.887237  0.112763
                    BTM        439         127  0.289294  0.710706
                    GBT        356          95  0.266854  0.733146
                    GVH        386          98  0.253886  0.746114
Metro/Non-Metro     BCO        669          82  0.122571  0.877429
                    BTR        422          47  0.111374  0.888626
                    BTS        677         175  0.258493  0.741507
                    KSG        496         112  0.225806  0.774194
                    NAC        365          66  0.180822  0.819178
Non-Metro/Metro     ADL        376          50  0.132979  0.867021
                    COB        455          38  0.083516  0.916484
                    GXK        400          44  0.110000  0.890000
                    PQB        448         108  0.241071  0.758929
                    TRB        510         142  0.278431  0.721569
Non-Metro/Non-Metro AVPWBBA    375          26  0.069333  0.930667
                    BAM        897         187  0.208473  0.791527
                    IBP        389          73  0.187661  0.812339
                    LAM        516          63  0.122093  0.877907
                    MOB        488         106  0.217213  0.782787
                    STC        443          49  0.110609  0.889391

In [17]:
# Imp_Routes
imp_route = list(set([i[1] for i in list(r_df.index)]))
print(len(imp_route))
data['Imp_Routes'] = np.where(data['Route'].isin(imp_route), data['Route'], 'Others')

20


In [18]:
# Vendors
vendor_df = data.groupby(['O_D_city_type','Vendor']).agg({'Vendor':'count','Delay_Flag':'sum'})
vendor_df['1'] = vendor_df['Delay_Flag']/vendor_df['Vendor']
vendor_df['0'] = 1 - vendor_df['1']
v_df = vendor_df.loc[(vendor_df['Vendor'] > 350) & (np.abs(vendor_df['1'] - vendor_df['0']) > 0.4)]
print(v_df.shape)
v_df

(25, 4)


Vendor  Delay_Flag         1         0
O_D_city_type       Vendor                                        
Metro/Metro         ABC10     1190         221  0.185714  0.814286
                    ABC124     540         383  0.709259  0.290741
                    ABC180     367         305  0.831063  0.168937
                    ABC49      389          14  0.035990  0.964010
Metro/Non-Metro     ABC15      376          42  0.111702  0.888298
                    ABC31      889         161  0.181102  0.818898
                    ABC51      537          92  0.171322  0.828678
Non-Metro/Metro     ABC12      755         135  0.178808  0.821192
                    ABC15      384          42  0.109375  0.890625
                    ABC31     1038         308  0.296724  0.703276
                    ABC34      425         392  0.922353  0.077647
                    ABC51      376         104  0.276596  0.723404
Non-Metro/Non-Metro ABC211     430          51  0.118605  0.881395
                    ABC251     890         187  0.210112  0.789888
                    ABC27      664         114  0.171687  0.828313
                    ABC36      384           4  0.010417  0.989583
                    ABC54      614         162  0.263844  0.736156
                    ABC57     1042         139  0.133397  0.866603
                    ABC58      423         367  0.867612  0.132388
                    ABC60     1471        1054  0.716519  0.283481
                    ABC69      487          53  0.108830  0.891170
                    ABC78      472          92  0.194915  0.805085
                    ABC85      409           5  0.012225  0.987775
                    ABC9       408          98  0.240196  0.759804
                    ABC97      951         136  0.143007  0.856993

In [19]:
# Imp_Vendors
imp_vendor = list(set([i[1] for i in list(v_df.index)]))
print(len(imp_vendor))
data['Imp_Vendors'] = np.where(data['Vendor'].isin(imp_vendor), data['Vendor'], 'Others')
# Vendor_city_type
imp_vendor_ct = list(set([i[1] + '_' + i[0] for i in list(v_df.index)]))
vendor_ct = (data['Vendor'] + '_' + data['O_D_city_type'])
data['Vendor_city_type'] = np.where(vendor_ct.isin(imp_vendor_ct), vendor_ct, 'Others')

22


In [719]:
# CVEHICLETYPE
vhtype_df = data.groupby(['Vendor','CVEHTYPE']).agg({'CVEHTYPE':'count','Delay_Flag':'sum'})
vhtype_df['1'] = vhtype_df['Delay_Flag']/vhtype_df['CVEHTYPE']
vhtype_df['0'] = 1 - vhtype_df['1']
vh_df = vhtype_df.loc[(vhtype_df['CVEHTYPE'] > 200) & (np.abs(vhtype_df['1'] - vhtype_df['0']) > 0.4)]
print(vh_df.shape)
vh_df.head()

(63, 4)


,,CVEHTYPE,Delay_Flag,1,0
Vendor,CVEHTYPE,,,,
ABC1,H24,293,52,0.177474,0.822526
ABC10,H24,1795,320,0.178273,0.821727
ABC100,M20,353,79,0.223796,0.776204
ABC12,H20,299,41,0.137124,0.862876
ABC124,MXL,424,307,0.724057,0.275943


In [720]:
#vhtype_df
imp_veh = list(set([i[1] for i in list(vh_df.index)]))
print(len(imp_veh))
data['Imp_CVEHTYPE'] = np.where(data['CVEHTYPE'].isin(imp_veh), data['CVEHTYPE'], 'Others')

12


In [721]:
def one_hot(df, cols):
    """
    @param df pandas DataFrame
    @param cols a list of columns to encode 
    @return a DataFrame with one-hot encoding
    """
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
        #df.drop([each],axis=1, inplace=True)
    return df

In [722]:
data.columns

Index(['Run Type', 'NCARNETNO', 'DFLIGHTDT', 'CFLIGHTNO', 'Route', 'CORG',
       'O_City_type', 'Org-Reg', 'CDST', 'D_City_type', 'Dst_Reg', 'Distance',
       'CVEHTYPE', 'TYPE', 'Vendor', 'Capacity', 'Consdwt', 'Pcs',
       'Util_conwt', 'Crit', 'NHIRE_AMT', 'FS', 'Incentive/Penalty',
       'NOTHERSAMT', 'NDISCOUNT', 'NBDEDEDUCT', 'Intra/Inter', 'CCONTTYPE',
       'Sch/NSch/Spl', 'Sch. Departure', 'Act. Departure', 'Sch. Arrival',
       'Actual Arrival', 'Expected Arrival', 'O_D', 'Departure delay',
       'Arrival Delay', 'day', 'month', 'year', 'date', 'Departure Year',
       'Actual Arrival Year', 'Delay', 'Delay_Flag', 'Month', 'Weekday',
       'Time_Interval', 'O_D_city_type', 'Imp_Routes', 'Imp_Vendors',
       'Vendor_city_type', 'Imp_CVEHTYPE'],
      dtype='object')

In [723]:
categorical_cols = ['Run Type','TYPE', 'Crit','Intra/Inter', 'CCONTTYPE',
       'Sch/NSch/Spl','O_D','Imp_Routes','Imp_Vendors','Imp_CVEHTYPE', 'O_D_city_type', 'Month', 'Weekday','Time_Interval']

In [724]:
print(data.shape)

(90076, 53)


In [725]:
data1 = one_hot(data, categorical_cols)
print(data1.shape)

(90076, 189)


In [726]:
drop_var = ['NHIRE_AMT','FS','Incentive/Penalty','NOTHERSAMT','NDISCOUNT',
            'NBDEDEDUCT','Departure Year','Actual Arrival Year','month']
data1.drop(drop_var,axis=1, inplace=True)

data1.shape

(90076, 180)

In [728]:
data1.columns.tolist()

['Run Type',
 'NCARNETNO',
 'DFLIGHTDT',
 'CFLIGHTNO',
 'Route',
 'CORG',
 'O_City_type',
 'Org-Reg',
 'CDST',
 'D_City_type',
 'Dst_Reg',
 'Distance',
 'CVEHTYPE',
 'TYPE',
 'Vendor',
 'Capacity',
 'Consdwt',
 'Pcs',
 'Util_conwt',
 'Crit',
 'Intra/Inter',
 'CCONTTYPE',
 'Sch/NSch/Spl',
 'Sch. Departure',
 'Act. Departure',
 'Sch. Arrival',
 'Actual Arrival',
 'Expected Arrival',
 'O_D',
 'Departure delay',
 'Arrival Delay',
 'day',
 'year',
 'date',
 'Delay',
 'Delay_Flag',
 'Month',
 'Weekday',
 'Time_Interval',
 'O_D_city_type',
 'Imp_Routes',
 'Imp_Vendors',
 'Vendor_city_type',
 'Imp_CVEHTYPE',
 'Run Type_AIRNET',
 'Run Type_SFC',
 'Run Type_SPEED',
 'TYPE_MXL',
 'TYPE_SXL',
 'Crit_40-50%',
 'Crit_50-60%',
 'Crit_60-70%',
 'Crit_<40%',
 'Crit_>70%',
 'Intra/Inter_Inter',
 'Intra/Inter_Intra',
 'CCONTTYPE_B',
 'CCONTTYPE_C',
 'CCONTTYPE_M',
 'Sch/NSch/Spl_Non-Sch Run',
 'Sch/NSch/Spl_Sch Run',
 'Sch/NSch/Spl_Spl Run',
 'O_D_EAST-EAST',
 'O_D_EAST-NORTH',
 'O_D_EAST-SOUTH1',
 'O_D_

In [729]:
data1.to_csv('D:/BlueDart/POC/Data_v5.csv', index = False)

In [444]:
pd.DataFrame(data1.Delay.describe())

,Delay
count,90076.000000
mean,1.119111
std,8.737856
min,-175.000000
25%,-2.000000
50%,0.000000
75%,3.000000
max,233.000000


In [ ]:
######################## EDA #######################

In [341]:
# NCARNETNO
# all unique

In [342]:
# DFLIGHTDT
min(data['DFLIGHTDT']), max(data['DFLIGHTDT'])

(Timestamp('2018-01-01 00:00:00'), Timestamp('2018-08-31 00:00:00'))

In [343]:
# CFLIGHTNO
print(len(set(data['CFLIGHTNO'])))

6622


In [344]:
# CORG and CDST
count_org = Counter(data['CORG'])
count_dest = Counter(data['CDST'])
print('Origin City', sorted(count_org.items(), key=lambda pair: pair[1], reverse=True)[:5])
print('Destination City', sorted(count_dest.items(), key=lambda pair: pair[1], reverse=True)[:5])

Origin City [('BHI', 10238), ('BTH', 6784), ('MHP', 6001), ('NHH', 5825), ('AMH', 4772)]
Destination City [('BHI', 8155), ('BTH', 6705), ('NHH', 5758), ('AMH', 5620), ('MHP', 4336)]


In [345]:
# City_type and City_type.1
count_org = Counter(data['D_City_type'])
count_dest = Counter(data['O_City_type'])
print('Origin City', sorted(count_org.items(), key=lambda pair: pair[1], reverse=True))
print('Destination City', sorted(count_dest.items(), key=lambda pair: pair[1], reverse=True))

Origin City [('Non-Metro', 49779), ('Metro', 40713)]
Destination City [('Metro', 45711), ('Non-Metro', 44781)]


In [346]:
# Org-Reg and Dst_Reg
count_org = Counter(data['Org-Reg'])
count_dest = Counter(data['Dst_Reg'])
print('Origin City', sorted(count_org.items(), key=lambda pair: pair[1], reverse=True))
print('Destination City', sorted(count_dest.items(), key=lambda pair: pair[1], reverse=True))

Origin City [('NORTH', 31338), ('WEST1', 20379), ('SOUTH2', 14418), ('SOUTH1', 9420), ('EAST', 8127), ('WEST2', 6810)]
Destination City [('NORTH', 30134), ('WEST1', 18455), ('SOUTH2', 16493), ('SOUTH1', 9239), ('EAST', 8577), ('WEST2', 7594)]


In [347]:
# Delay
df = pd.DataFrame()
df['x'] = data['Actual Arrival'] - data['Sch. Arrival']
df['y'] = data['Act. Departure'] - data['Sch. Departure']
df['z'] = data['Actual Arrival'] - data['Expected Arrival']
df['a'] = df.y + df.z
df.loc[df.x != df.a, :].shape

(0, 4)

In [361]:
# Intra/Inter
print(Counter(data['Intra/Inter']))
pd.crosstab(index=data['Delay_Flag'],columns=data['Intra/Inter']).apply(lambda x: x/x.sum(), axis=0)

Counter({'Intra': 51983, 'Inter': 38509})


Intra/Inter,Inter,Intra
Delay_Flag,,
0,0.501519,0.622819
1,0.498481,0.377181


In [362]:
# Sch/NSch/Spl
print(Counter(data['Sch/NSch/Spl']))
pd.crosstab(index=data['Delay_Flag'],columns=data['Sch/NSch/Spl']).apply(lambda x: x/x.sum(), axis=0)

Counter({'Sch Run': 55936, 'Non-Sch Run': 28720, 'Spl Run': 5836})


Sch/NSch/Spl,Non-Sch Run,Sch Run,Spl Run
Delay_Flag,,,
0,0.56257,0.587976,0.452879
1,0.43743,0.412024,0.547121


In [363]:
# Vendor
print('Unique Vendors:',len(set(data['Vendor'])))
top_vendors = list(map(lambda x: x[0], sorted(Counter(data['Vendor']).items(),key=lambda pair: pair[1], reverse=True)[:10]))
data['Top_Vendors'] = [v if v in top_vendors else 'Others' for v in data['Vendor']]
pd.crosstab(index=data['Delay_Flag'],columns=data['Top_Vendors']).apply(lambda x: x/x.sum(), axis=0)

Unique Vendors: 369


Top_Vendors,ABC10,ABC12,ABC233,ABC26,ABC31,ABC46,ABC5,ABC6,ABC7,ABC8,Others
Delay_Flag,,,,,,,,,,,
0,0.814735,0.669548,0.61686,0.678878,0.762154,0.496811,0.59306,0.389555,0.470103,0.414105,0.584167
1,0.185265,0.330452,0.38314,0.321122,0.237846,0.503189,0.40694,0.610445,0.529897,0.585895,0.415833


In [364]:
# Utilization
# Utilization = Weight/Capacity
data_util = data[['Consdwt','Capacity','Util_conwt']].copy()
data_util['Calc_Util'] = data['Consdwt']/data['Capacity']

# pd.to_numeric(data_util['Util_conwt'])
# data_util['Calc_Util']
data_util['Util_conwt'] = data_util['Util_conwt'].str.rstrip('%').astype('float') / 100.0
np.max(data_util['Util_conwt'] - data_util['Calc_Util'])

0.03999999999999204

In [365]:
# Utilization Category
print(Counter(data['Crit']))
pd.crosstab(index=data['Delay_Flag'],columns=data['Crit']).apply(lambda x: x/x.sum(), axis=0)

Counter({'>70%': 54458, '<40%': 17372, '60-70%': 7647, '50-60%': 6040, '40-50%': 4975})


Crit,40-50%,50-60%,60-70%,<40%,>70%
Delay_Flag,,,,,
0,0.629548,0.625993,0.593697,0.657955,0.528958
1,0.370452,0.374007,0.406303,0.342045,0.471042


In [366]:
# Run Type
pd.crosstab(index=data['Delay_Flag'],columns=data['Run Type']).apply(lambda x: x/x.sum(), axis=0)

Run Type,AIRNET,SFC,SPEED
Delay_Flag,,,
0,0.732166,0.58252,0.385866
1,0.267834,0.41748,0.614134


In [369]:
# Pcs
(data['Pcs'] == 0)

382

In [379]:
((data['Consdwt'] == 0)).sum()

415

In [368]:
# Distance(kms)
print('Runs with 0 distance:',data.loc[data['Distance'] == 0,['CORG', 'CDST']].shape[0])
print(data['Distance'].describe())

Runs with 0 distance: 0
count    90492.000000
mean       724.660056
std        663.763875
min          1.000000
25%        220.000000
50%        439.000000
75%       1130.000000
max       2905.000000
Name: Distance, dtype: float64


In [812]:
# # WoE

# def woe(df, var):
    
#     df_woe = df.groupby([var]).agg({var:'count','Delay_Flag':'sum'})
#     df_woe = df_woe[[var,'Delay_Flag']]
#     df_woe.columns = ['count','#1']
#     df_woe['Feature'] = var
#     df_woe = df_woe.reset_index()
#     df_woe = df_woe[['Feature',var,'count','#1']]
#     df_woe.columns = ['Feature','Feature Value','count','#1']
#     df_woe['%1s'] = df_woe['#1']/df_woe['count']
#     df_woe['%0s'] = 1 - df_woe['%1s']
#     df_woe['WoE'] = np.log(df_woe['%0s']/df_woe['%1s'])
#     var = var.replace('/','_')
#     return df_woe

# cat_var = ['Run Type', 'Route', 'CVEHTYPE', 'TYPE', 'Vendor','Crit','Intra/Inter', 'CCONTTYPE',
#        'Sch/NSch/Spl', 'O_D', 'Month', 'Weekday',
#        'Time_Interval', 'O_D_city_type']
# df_woe = pd.DataFrame()
# for var in cat_var:
#     df_woe = df_woe.append(woe(data.copy(),var))
    
# df_woe.to_csv('D:/BlueDart/POC/Features_WoE.csv', index = False)

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:12: RuntimeWarning: divide by zero encountered in log
